<a href="https://colab.research.google.com/github/LeoIba/Cursos-Infor-San-Luis-2025/blob/main/Copia_llms_kgs_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="right"><i>Matías Torres Esteban<br>Diciembre, 2025</i></div>

# Interpretación de Textos con LLMs

Los grandes modelos de lenguaje (LLMs) son conocidos por su capacidad de aprender patrones complejos de los textos y de resolver en consecuencia varias tareas de procesamiento del lenguaje natural, como lo son la traducción automática, la generación de resúmenes y la resolución de preguntas de dominios especializados. Es por esto que han surgido nuevas lineas de investigación que buscan utilizar a los grandes modelos de lenguaje (LLMs) como *interpretadores semánticos de textos*. Su objetivo es representar el significado de un texto escrito en lenguaje natural en uno o más lenguajes lógicos —por ejemplo, la lógica proposicional o la lógica de primer orden— para explicitar la información contenida allí y brindarle mayor estructura. De esta manera, podemos obtener representaciones avanzadas del significado de un texto que sean más faciles de almacenar en una computadora y puedan ser manipuladas algorítmicamente.

Un lenguaje lógico que podemos utilizar para representar la información contenida en un texto es el lenguaje de los *Grafos de Conocimiento (KGs)*, los cuales permiten codificar hechos y proposiciones del mundo mediante ternas $(f,r,o)$ donde $f$ denota un concepto fuente, $r$ una relación semántica y $o$ un concepto objetivo. Así, la información contenida en el siguiente texto:

* *"El Covid-19 es una enfermedad infecciosa causada por el SARS-CoV-2. Produce síntomas que incluyen fiebre, tos y fatiga"*,

puede representarse como la siguiente colección de ternas de conocimiento:

* *(Covid-19, es una, Enfermedad Infecciosa)*
* *(Covid-19, causada por, SARS-CoV-2)*.
* *(Covid-19, tiene síntoma, Fiebre)*.
* *(Covid-19, tiene síntoma, Tos)*.
* *(Covid-19, tiene síntoma, Fatiga)*.

Esta colección de ternas puede representase como un grafo dirigido y etiquetado, como se muestra en la siguiente figura:

![GrafoConocimiento](https://raw.githubusercontent.com/matizzat/InforSanLuis-2025-LLMs/5596c0c724cd8e63742866ad998479ecd6f685d2/imagenes/grafo_conocimiento_covid19.svg)

Vemos que este tipo de representación es más rica que el texto puro porque explicita los conceptos y relaciones más relevantes. De esta manera, la interpretación y el análisis de la información extraída se vuelven más accesibles tanto para un usuario como para una computadora, y además se facilita su almacenamiento en bases de datos. Finalmente, si pudiéramos procesar automáticamente un gran conjunto de documentos de un dominio especializado —como la biología o la medicina— y transformarlos en grafos de conocimiento, podríamos aplicar todo el aparato matemático de la Teoría de Grafos para analizar estos sistemas conceptuales y revelar información del dominio que está escondida en los textos.

En esta notebook utilizaremo al modelo Gemini junto a código Python para crear automáticamente grafos de conocimiento a partir de textos. Este ejercicio nos enseñará a coordinar diferentes invocaciones al modelo y a escribir correctamente nuestros prompts para sacarle el máximo provecho posible.

## Proceso de Interpretación

Vamos a implementar un procedimiento de interpretación textual inspirado en la metodología propuesta por Joseph Novak para la construcción de mapas conceptuales [1]. Este procedimiento permite generar un KG a partir de un texto expositivo mediante una secuencia estructurada de 4 pasos:

1. Solicitamos al modelo que analice el texto ``<texto>`` y genere una pregunta de enfoque ``<pregunta>``. Las ternas de conocimiento generadas en los
próximos pasos deberı́an ayudar a responder esta pregunta.

2. Solicitamos al modelo que analice ``<texto>`` y ``<pregunta>`` y que luego genere una lista de conceptos ``<conceptos>``. Los conceptos extraı́dos deben
estar explicitamente mencionados en el texto y tienen que ayudar a responder la pregunta de enfoque.

3. Solictamos al modelo que analice ``<texto>``, ``<pregunta>`` y ``<conceptos>`` y que genere una lista de relaciones semánticas ``<relaciones>``.

4. Solicitamos al modelo que analice ``<texto>``, ``<pregunta>``, ``<conceptos>`` y ``<relaciones>`` en conjunto y que luego genere una lista de ternas de conocimiento ``<ternas>``.

En cada paso realizamos un procesamiento de las etiquetas de conceptos y relaciones para convertir todos sus caracteres a minúsculas y eliminar espacios en blanco innecesarios.

## Código

**Advertencia:** Si la API de Gemini no está disponible en este momento pueden simular el proceso en ChatGPT o el chat de Gemini y reintentar ejecutar el código más tarde.

Primero extraemos los recursos desde Github (textos y prompts):

In [1]:
!git clone https://github.com/matizzat/InforSanLuis-2025-LLMs
%cd InforSanLuis-2025-LLMs

Cloning into 'InforSanLuis-2025-LLMs'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 71 (delta 27), reused 33 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 628.54 KiB | 6.35 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/InforSanLuis-2025-LLMs


Instalamos la librería Pyvis para visualizar grafos de conocimiento:

In [2]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.3 MB/s eta 0:00:00


Importamos las librerías necesarias

In [3]:
from pyvis.network import Network
from google.colab import userdata
from pyvis import network as net
from google.genai import types
from google import genai
from typing import List
import networkx as nx
import pprint
import json
import re

Instanciamos un cliente para invocar al modelo Gemini. Para ejecutar esta celda deben obtener una clave del siguiente [enlace](https://ai.google.dev/gemini-api/docs/api-key). También tienen que configurar Google Colab para utilizar esta clave (Ver  [Tutorial](https://www.youtube.com/watch?v=snrvP_TZjvw))

In [13]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') #La clave la obtiene de los Secretos
cliente_genai = genai.Client(api_key=GOOGLE_API_KEY)

Definimos una función auxiliar para abrir el contenido de un archivo y cargamos las instrucciones con las que invocaremos al modelo. En cada invocación nosotros le proveemos al modelo una **instrucción de sistema** y una **instrucción de usuario**:

* En la instrucción de sistema nosotros le especificamos al modelo cual es el rol que debe cumplir, le explicamos en términos generales la tarea a resolver y le brindamos ejemplos concretos de cómo hay que resolverla y cual es el formato de salida esperado.

* En la instrucción de usuario especificamos cual es el texto corriente que queremos analizar.

La técnica donde le damos ejemplos concretos al modelo de cómo debe resolver una tarea se denomina **aprendizaje en contexto** y es muy importante aprenderla para embeber a los LLMs en procesos automáticos y aplicarlos en dominios especializados.

In [12]:
def abrir_contenido_archivo(nombre_archivo: str): #Función General para abrir Archivos y devolverlos como String
    with open(nombre_archivo, 'r') as f:
        return f.read()

crear_pregunta_sistema   = abrir_contenido_archivo('./instrucciones/crear_pregunta_sistema.txt')
crear_conceptos_sistema  = abrir_contenido_archivo('./instrucciones/crear_conceptos_sistema.txt')
crear_relaciones_sistema = abrir_contenido_archivo('./instrucciones/crear_relaciones_sistema.txt')
crear_ternas_sistema     = abrir_contenido_archivo('./instrucciones/crear_ternas_sistema.txt')

crear_pregunta_usuario   = abrir_contenido_archivo('./instrucciones/crear_pregunta_usuario.txt')
crear_conceptos_usuario  = abrir_contenido_archivo('./instrucciones/crear_conceptos_usuario.txt')
crear_relaciones_usuario = abrir_contenido_archivo('./instrucciones/crear_relaciones_usuario.txt')
crear_ternas_usuario     = abrir_contenido_archivo('./instrucciones/crear_ternas_usuario.txt')

Ejecuten la siguiente celda probando distintos valores para visualizar las instrucciones con las que vamos a invocar al modelo. Observar que las instrucciones del usuario funcionan como plantillas: las completamos dinámicamente a medida que el LLM avance en cada paso del proceso. Los espacios a modificar dinámicamente están encerrados entre llaves ``{`` y ``}``.

In [14]:
print(crear_conceptos_sistema)

Eres un creador de mapas conceptuales que analiza textos científicos y extrae de ellos los conceptos y relaciones más importantes. Un concepto es un patrón o regularidad en objetos, eventos o registros de objetos y eventos, designado con una etiqueta. Los conceptos se relacionan entre sí mediante frases significativas que forman proposiciones. Te daré un texto de conocimiento delimitado por @ con la siguiente estructura:
    1. Pregunta Focal: Una pregunta que debes responder enumerando los conceptos relevantes.
    2. Conocimiento: Un texto que responde la pregunta focal utilizando conceptos relevantes. Los conceptos que extraigas deben estar explícitamente mencionados o derivados de este texto.
Tu tarea es escribir una lista de conceptos que luego utilizaremos para responder la pregunta focal. Los conceptos deben estar explícitamente mencionados o derivados del Texto de Conocimiento. Incluye los conceptos mencionados en la Pregunta Focal.

Ejemplos:

Texto de Conocimiento:
@
Pregunta

A continuación definimos funciones auxiliares que necesitaremos en nuestro proceso:
* ``invocar_llm`` recibe como parámetros una instrucción de sistema y una instrucción de usuario y con ellos invoca al modelo Gemini 2.5 Flash. La invocación esta fijada con un valor de temperatura bajo para que las respuestas del modelo sean determinísticas y no varíen en diferentes invocaciones.  
* ``abrir_lote_de_documentos`` se encarga de abrir una lista de documentos desde la ruta especificada. Cada documento posee un título y un texto.
* ``guardar_mapa_conceptuales``: Almacena una lista de mapas conceptuales en un archivo JSON. Cada mapa conceptual está codificado como un diccionario que tiene los siguientes elementos:
  * ``titulo``: Un string con el título del mapa conceptual para identificarlo.
  * ``pregunta``: Un string que representa la pregunta focal.
  * ``conceptos``: Una lista de strings que representa las etiquetas de conceptos.
  * ``relaciones``: Una lista de strings que representa las etiquetas de las relaciones semánticas.
  * ``ternas``. Una lista de ternas de conocimiento, donde cada una es un diccionario con las componentes `f` (concepto fuente), `r` (relación) y `o` (concepto objetivo).
* ``dibujar_mapas_conceptuales``: Dibuja un conjunto de mapas conceptuales como un único grafo dirigido y etiquetado, el cual se almacena en un archivo HTML.

In [15]:
def invocar_llm(sistema: str, usuario: str):
    """
    Invoca al modelo de lenguaje Gemini 2.5 Flash.

    Parámetros:
        sistema (str): Instrucciones del sistema para el modelo.
        usuario  (str): Consulta o instrucciones del usuario.

    Retorna:
        str: Texto generado por el modelo.
    """
    global cliente_genai

    respuesta = cliente_genai.models.generate_content(
        config = types.GenerateContentConfig(
            system_instruction = sistema,
            temperature = 0.1
        ),
        model = "gemini-2.5-flash",
        contents = usuario
    )

    return respuesta.text


def abrir_lote_de_documentos(ruta_entrada: str) -> list[dict]:
    """
    Abre un lote de documentos desde un archivo JSON.

    Parámetros:
        ruta_entrada (str): Ruta del archivo que contiene los documentos.

    Retorna:
        list[dict]: Lista de documentos.
    """
    with open(ruta_entrada, 'r') as f:
        textos = json.load(f)
    return textos


def guardar_mapas_conceptuales(ruta_salida: str, mapas: dict):
    """
    Guarda una lista de mapas conceptuales en un archivo JSON.

    Parámetros:
        ruta_salida (str): Ruta del archivo de salida.
        mapas (dict): Mapas conceptuales a guardar.
    """
    with open(ruta_salida, 'w') as f:
        json.dump(mapas, f, indent=4)


def dibujar_mapas_conceptuales(ruta_salida: str, mapas: dict):
    """
    Crea una visualización pyvis de una lista de mapas conceptuales
    y la guarda como archivo HTML.

    Parámetros:
        mapas (dict): Lista de mapas conceptuales.
        ruta_salida (str): Archivo HTML donde se guardará la visualización.
    """
    G = nx.MultiDiGraph()

    for mapa in mapas:
        pregunta_focal = mapa['pregunta']
        ternas = mapa['ternas']

        for terna in ternas:
            f = terna['f']
            r = terna['r']
            o = terna['o']

            if f not in G:
                G.add_node(f, label=f, color='gray')
            if o not in G:
                G.add_node(o, label=o, color='gray')

            G.add_edge(
                f,
                o,
                label=r,
                title=pregunta_focal,
                color='black'
            )

    vis = net.Network(directed=True)
    vis.from_nx(G)
    vis.save_graph(ruta_salida)


Abrimos el lote de documentos y mostramos el texto del primer documento que está almacenado allí. Si revisan el archivo `documentos.json` verán que cada documento consiste de un título y un texto.

In [16]:
documentos = abrir_lote_de_documentos('./datos/documentos.json')
texto = documentos[0]['texto']
print(texto)

El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
El término agua, generalmente, se refiere a la sustancia en su estado líquido, aunque esta puede hallarse en su forma sólida, llamada hielo, y en su forma gaseosa, denominada vapor.
Es una sustancia bastante común en la Tierra y el sistema solar, donde se encuentra principalmente en forma de vapor o de hielo.
Es indispensable para el origen y sustento de la vida.



Definimos una función auxiliar para normalizar la etiqueta de un concepto o una relación semántica. Esta elimina el carácter especial `@`, quita los espacios en blanco innecesarios y convierte todas las letras a minúsculas

In [17]:
def normalizar_etiqueta(etiqueta: str):
    etiqueta = etiqueta.replace('@','')
    etiqueta = etiqueta.strip()
    etiqueta = " ".join(etiqueta.split())
    etiqueta = etiqueta.lower()
    return etiqueta

Formateamos el prompt de usuario para instruirle al modelo que genere una pregunta de enfoque.

In [42]:
instruccion_crear_pregunta_formateada = crear_pregunta_usuario.format(texto = texto)
print(instruccion_crear_pregunta_formateada )

Texto de Conocimiento:
@
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
El término agua, generalmente, se refiere a la sustancia en su estado líquido, aunque esta puede hallarse en su forma sólida, llamada hielo, y en su forma gaseosa, denominada vapor.
Es una sustancia bastante común en la Tierra y el sistema solar, donde se encuentra principalmente en forma de vapor o de hielo.
Es indispensable para el origen y sustento de la vida.

@
Pregunta Focal:



Invocamos al modelo de lenguaje y le solicitamos que cree una pregunta focal para el texto dado.

In [23]:
pregunta_focal = invocar_llm(
    usuario = instruccion_crear_pregunta_formateada,
    sistema = crear_pregunta_sistema)

pregunta_focal = normalizar_etiqueta(pregunta_focal)
print(pregunta_focal)

¿qué es el agua, cuál es su composición, sus estados y su importancia?


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de conceptos.

In [20]:
instruccion_crear_conceptos_formateada = crear_conceptos_usuario.format(pregunta = pregunta_focal, texto = texto)
print(instruccion_crear_conceptos_formateada)

Texto de Conocimiento:
@
Pregunta Focal:
¿qué es el agua, cuál es su composición, sus estados y su importancia?
Conocimiento:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
El término agua, generalmente, se refiere a la sustancia en su estado líquido, aunque esta puede hallarse en su forma sólida, llamada hielo, y en su forma gaseosa, denominada vapor.
Es una sustancia bastante común en la Tierra y el sistema solar, donde se encuentra principalmente en forma de vapor o de hielo.
Es indispensable para el origen y sustento de la vida.

@
Lista de Conceptos:



Invocamos al modelo de lenguaje para que genere una lista de conceptos a partir del texto y la pregunta de enfoque:

In [24]:
respuesta_llm = invocar_llm(
    usuario = instruccion_crear_conceptos_formateada,
    sistema = crear_conceptos_sistema)

print(respuesta_llm)

Lista de Conceptos:
Agua
Sustancia
Molécula
Átomos de hidrógeno
Átomo de oxígeno
H2O
Enlace covalente
Estado líquido (del agua)
Estado sólido (del agua)
Hielo
Estado gaseoso (del agua)
Vapor
Tierra
Sistema solar
Indispensable para la vida
Origen de la vida
Sustento de la vida


Convertimos el texto extraido en una lista de Python:

In [46]:
# Expresión regular para extraer una lista de conceptos:
conceptos_exp_reg = r'[\w\d].*?\n|[\w\d].*$'

lista_conceptos = re.findall(conceptos_exp_reg, respuesta_llm)
lista_conceptos = [normalizar_etiqueta(concepto) for concepto in lista_conceptos]

pprint.pprint(lista_conceptos)

['agua es una sustancia !',
 'h2o está compuesta por átomos de hidrógeno !',
 'h2o está compuesta por átomo de oxígeno !',
 'átomos de hidrógeno unidos por un enlace covalente !',
 'átomo de oxígeno unidos por un enlace covalente !',
 'agua se refiere a la sustancia en su estado líquido (del agua) !',
 'agua puede hallarse en su forma estado sólido (del agua) !',
 'estado sólido (del agua) llamada hielo !',
 'agua puede hallarse en su forma estado gaseoso (del agua) !',
 'estado gaseoso (del agua) denominada vapor !',
 'agua es común en tierra !',
 'agua es común en sistema solar !',
 'agua se encuentra en forma de vapor !',
 'agua se encuentra en forma de hielo !',
 'agua es indispensable para el origen de la vida !',
 'agua es indispensable para el sustento de la vida !']


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de relaciones semánticas:

In [27]:
instruccion_crear_relaciones_formateada = crear_relaciones_usuario.format(
    pregunta = pregunta_focal,
    conceptos = "\n".join(lista_conceptos),
    texto = texto)
print(instruccion_crear_relaciones_formateada)

Texto de Conocimiento:
@
Pregunta Focal:
¿qué es el agua, cuál es su composición, sus estados y su importancia?
Lista de Conceptos:
lista de conceptos:
agua
sustancia
molécula
átomos de hidrógeno
átomo de oxígeno
h2o
enlace covalente
estado líquido (del agua)
estado sólido (del agua)
hielo
estado gaseoso (del agua)
vapor
tierra
sistema solar
indispensable para la vida
origen de la vida
sustento de la vida
Conocimiento:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
El término agua, generalmente, se refiere a la sustancia en su estado líquido, aunque esta puede hallarse en su forma sólida, llamada hielo, y en su forma gaseosa, denominada vapor.
Es una sustancia bastante común en la Tierra y el sistema solar, donde se encuentra principalmente en forma de vapor o de hielo.
Es indispensable para el origen y sustento de la vida.

@
Relaciones Semánticas:



Invocamos al modelo de lenguaje para que genere una lista de relaciones a partir del texto, la pregunta de enfoque y la lista de conceptos:

In [28]:
respuesta_llm = invocar_llm(
    sistema = crear_relaciones_sistema,
    usuario = instruccion_crear_relaciones_formateada)

print(respuesta_llm)

Relaciones Semánticas:
Es una
Está compuesta por
Unidos por un
Se refiere a la sustancia en su
Puede hallarse en su forma
Llamada
Denominada
Es común en
Se encuentra en forma de
Es indispensable para el


Convertimos el texto extraido en una lista de Python:

In [29]:
# Expresión regular para extraer una lista de relaciones semánticas:
relaciones_exp_reg = r'[\w\d].*?\n|[\w\d].*$'

lista_relaciones = re.findall(relaciones_exp_reg, respuesta_llm)
lista_relaciones = [normalizar_etiqueta(relacion) for relacion in lista_relaciones]

pprint.pprint(lista_relaciones)

['relaciones semánticas:',
 'es una',
 'está compuesta por',
 'unidos por un',
 'se refiere a la sustancia en su',
 'puede hallarse en su forma',
 'llamada',
 'denominada',
 'es común en',
 'se encuentra en forma de',
 'es indispensable para el']


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de ternas de conocimiento:

In [30]:
instruccion_crear_ternas_formateada = crear_ternas_usuario.format(
    pregunta = pregunta_focal,
    conceptos = "\n".join(lista_conceptos),
    relaciones = "\n".join(lista_relaciones),
    texto = texto)

print(instruccion_crear_ternas_formateada)

Texto de Conocimiento:
@
Pregunta Focal:
¿qué es el agua, cuál es su composición, sus estados y su importancia?
Lista de Conceptos:
lista de conceptos:
agua
sustancia
molécula
átomos de hidrógeno
átomo de oxígeno
h2o
enlace covalente
estado líquido (del agua)
estado sólido (del agua)
hielo
estado gaseoso (del agua)
vapor
tierra
sistema solar
indispensable para la vida
origen de la vida
sustento de la vida
Lista de Relaciones:
relaciones semánticas:
es una
está compuesta por
unidos por un
se refiere a la sustancia en su
puede hallarse en su forma
llamada
denominada
es común en
se encuentra en forma de
es indispensable para el
Conocimiento:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
El término agua, generalmente, se refiere a la sustancia en su estado líquido, aunque esta puede hallarse en su forma sólida, llamada hielo, y en su forma gaseosa, denominada vapor.
Es una sustancia bastante común en

Invocamos al modelo de lenguaje para que genere una lista de ternas de conocimiento  a partir del texto, la pregunta de enfoque, la lista de conceptos y la lista de relaciones semánticas:

In [31]:
respuesta_llm = invocar_llm(
    sistema = crear_ternas_sistema,
    usuario = instruccion_crear_ternas_formateada)

print(respuesta_llm)

@! agua @ es una @ sustancia !@
@! h2o @ está compuesta por @ átomos de hidrógeno !@
@! h2o @ está compuesta por @ átomo de oxígeno !@
@! átomos de hidrógeno @ unidos por un @ enlace covalente !@
@! átomo de oxígeno @ unidos por un @ enlace covalente !@
@! agua @ se refiere a la sustancia en su @ estado líquido (del agua) !@
@! agua @ puede hallarse en su forma @ estado sólido (del agua) !@
@! estado sólido (del agua) @ llamada @ hielo !@
@! agua @ puede hallarse en su forma @ estado gaseoso (del agua) !@
@! estado gaseoso (del agua) @ denominada @ vapor !@
@! agua @ es común en @ tierra !@
@! agua @ es común en @ sistema solar !@
@! agua @ se encuentra en forma de @ vapor !@
@! agua @ se encuentra en forma de @ hielo !@
@! agua @ es indispensable para el @ origen de la vida !@
@! agua @ es indispensable para el @ sustento de la vida !@


Extraemos las ternas de conocimiento del modelo:

In [33]:
#Para diferenciar entre componente de ternas, se usa el @
#Para marcar el inicio de una nueva terna se usa el @
ternas_exp_reg = r'@! (.+?) @ (.+?) @ (.+?) !@'

L = re.findall(ternas_exp_reg, respuesta_llm)

lista_ternas = []

for f, r, o in L:
    lista_ternas.append({
        'f': normalizar_etiqueta(f),
        'r': normalizar_etiqueta(r),
        'o': normalizar_etiqueta(o)
    })

pprint.pprint(lista_ternas)

[{'f': 'agua', 'o': 'sustancia', 'r': 'es una'},
 {'f': 'h2o', 'o': 'átomos de hidrógeno', 'r': 'está compuesta por'},
 {'f': 'h2o', 'o': 'átomo de oxígeno', 'r': 'está compuesta por'},
 {'f': 'átomos de hidrógeno', 'o': 'enlace covalente', 'r': 'unidos por un'},
 {'f': 'átomo de oxígeno', 'o': 'enlace covalente', 'r': 'unidos por un'},
 {'f': 'agua',
  'o': 'estado líquido (del agua)',
  'r': 'se refiere a la sustancia en su'},
 {'f': 'agua',
  'o': 'estado sólido (del agua)',
  'r': 'puede hallarse en su forma'},
 {'f': 'estado sólido (del agua)', 'o': 'hielo', 'r': 'llamada'},
 {'f': 'agua',
  'o': 'estado gaseoso (del agua)',
  'r': 'puede hallarse en su forma'},
 {'f': 'estado gaseoso (del agua)', 'o': 'vapor', 'r': 'denominada'},
 {'f': 'agua', 'o': 'tierra', 'r': 'es común en'},
 {'f': 'agua', 'o': 'sistema solar', 'r': 'es común en'},
 {'f': 'agua', 'o': 'vapor', 'r': 'se encuentra en forma de'},
 {'f': 'agua', 'o': 'hielo', 'r': 'se encuentra en forma de'},
 {'f': 'agua', 'o':

Unimos todas las componentes obtenidas en un único diccionario Python que representa el grafo de conocimiento (o mapa conceptual):

In [37]:
mapa_conceptual = {'titulo': documentos[0]['titulo'], 'pregunta': pregunta_focal, 'ternas': lista_ternas, 'conceptos': lista_conceptos, 'relaciones': lista_relaciones}
pprint.pprint(mapa_conceptual)

{'conceptos': ['lista de conceptos:',
               'agua',
               'sustancia',
               'molécula',
               'átomos de hidrógeno',
               'átomo de oxígeno',
               'h2o',
               'enlace covalente',
               'estado líquido (del agua)',
               'estado sólido (del agua)',
               'hielo',
               'estado gaseoso (del agua)',
               'vapor',
               'tierra',
               'sistema solar',
               'indispensable para la vida',
               'origen de la vida',
               'sustento de la vida'],
 'pregunta': '¿qué es el agua, cuál es su composición, sus estados y su '
             'importancia?',
 'relaciones': ['relaciones semánticas:',
                'es una',
                'está compuesta por',
                'unidos por un',
                'se refiere a la sustancia en su',
                'puede hallarse en su forma',
                'llamada',
                'denominada',
  

Almacenamos en un archivo JSON el grafo de conocimiento obtenido y lo visalizamos en un archivo HTML 😀. Para ver el grafo descarguen el archivo `mapas.html` y abranlo en una nueva pestaña de su navegador:

In [38]:
guardar_mapas_conceptuales('./mapas.json', [mapa_conceptual])
dibujar_mapas_conceptuales('./mapas.html', [mapa_conceptual])

# Tarea

La tarea consiste en diseñar e implementar una estrategia propia para la creación de mapas conceptuales utilizando el modelo de lenguaje Gemini. Todo el proceso debe integrarse en una función llamada crear_mapa_conceptual, la cual recibe como entrada un texto y devuelve como salida un diccionario de Python que codifica el mapa conceptual correspondiente.

A partir del archivo `documentos.json`, deberán generar un único mapa conceptual por cada texto, y almacenar todos ellos en un único archivo JSON `mapas.json`, siguiendo el formato ilustrado en el ejemplo anterior.

Se recomienda experimentar con distintas estrategias de prompting y diferentes algoritmos. Pueden usar como referencia los prompts y piezas de código presentados en la sección previa. Además, el siguiente recurso puede serles útil como guía para diseñar prompts:
https://www.promptingguide.ai/

**Consejo**:
* Incorporen un paso adicional en la estrategia anterior que le pida al LLM mejorar un mapa conceptual previamente generado.

---

### Entrega

Cuando finalicen, suban su notebook, los prompts utilizados y todos los archivos con los mapas conceptuales generados a un repositorio público de GitHub. Envíen el enlace del repositorio al correo mat.torreta@gmail.com con el asunto:

* *InforSanLuis25-LLMs Entrega*

En el cuerpo del correo deben incluir:

* Nombre completo

* DNI

La entrega deberá realizarse antes del viernes 05 de diciembre a las 23:59 para aprobar el curso.

---

### Método a implementar

Implementen la mayor parte de su estrategia de creación de mapas conceptuales en el cuerpo de esta función. Pueden crear sus propias funciones auxiliares e invocarlas desde aquí si lo necesitan.

In [57]:

def normalizar_etiquetaNew(etiqueta: str):
    etiqueta = etiqueta.replace('@','')
    etiqueta = etiqueta.strip()
    etiqueta = " ".join(etiqueta.split())
    return etiqueta

def crear_mapa_conceptual(string: titulo, texto: texto) -> dict:
    #El texto recibido es el documento completo sin título del que debemos crear un mapa conceptual
    #         ORDEN GENERAL
    # 1. Crear la Pregunta/Prompt
    instruccion_crear_pregunta_formateada = crear_pregunta_usuario.format(texto = texto)
    # 2. Obtener la Pregunta Focal
    pregunta_focal = invocar_llm(usuario = instruccion_crear_pregunta_formateada, sistema = crear_pregunta_sistema)
    pregunta_focal = normalizar_etiqueta(pregunta_focal)
    # 3. Crear los Conceptos a partir del Texto
    instruccion_crear_conceptos_formateada = crear_conceptos_usuario.format(pregunta = pregunta_focal, texto = texto)
    respuesta_llm = invocar_llm(usuario = instruccion_crear_conceptos_formateada, sistema = crear_conceptos_sistema)
    conceptos_exp_reg = r'[\w\d].*?\n|[\w\d].*$'
    lista_conceptos = re.findall(conceptos_exp_reg, respuesta_llm)
    lista_conceptos = [normalizar_etiquetaNew(concepto) for concepto in lista_conceptos]
    # 4. Crear las Relaciones a partir del Texto
    instruccion_crear_relaciones_formateada = crear_relaciones_usuario.format(pregunta = pregunta_focal,conceptos = "\n".join(lista_conceptos), texto = texto)
    relaciones_exp_reg = r'[\w\d].*?\n|[\w\d].*$'
    lista_relaciones = re.findall(relaciones_exp_reg, respuesta_llm)
    lista_relaciones = [normalizar_etiqueta(relacion) for relacion in lista_relaciones]
    # 5. Crear las Ternas a partir del Texto
    instruccion_crear_ternas_formateada = crear_ternas_usuario.format(pregunta = pregunta_focal,conceptos = "\n".join(lista_conceptos),relaciones = "\n".join(lista_relaciones),texto = texto)
    respuesta_llm = invocar_llm(
    sistema = crear_ternas_sistema,
    usuario = instruccion_crear_ternas_formateada)
    ternas_exp_reg = r'@! (.+?) @ (.+?) @ (.+?) !@'
    L = re.findall(ternas_exp_reg, respuesta_llm)
    lista_ternas = []
    for f, r, o in L:
        lista_ternas.append({
            'f': normalizar_etiquetaNew(f),
            'r': normalizar_etiqueta(r),
            'o': normalizar_etiquetaNew(o)
        })

    mapa_conceptual = {'titulo': titulo, 'pregunta': pregunta_focal, 'ternas': lista_ternas, 'conceptos': lista_conceptos, 'relaciones': lista_relaciones}
    #GUARDAR EL MAPA y DIBUJARLO
    guardar_mapas_conceptuales('./mapaNew.json', [mapa_conceptual])
    dibujar_mapas_conceptuales('./mapaNew.html', [mapa_conceptual])

    return mapa_conceptual

    raise RuntimeError("Función no implementada.")

documentos = abrir_lote_de_documentos('./datos/documentos.json')
texto = documentos[1]['texto']
titulo = documentos[1]['titulo']
mapa_conceptual = crear_mapa_conceptual(titulo, texto)
mapa_conceptual

{'titulo': 'don_quijote',
 'pregunta': '¿qué es don quijote de la mancha y cuáles son sus principales características?',
 'ternas': [{'f': 'Don Quijote de la Mancha', 'r': 'es una', 'o': 'Novela'},
  {'f': 'Don Quijote de la Mancha',
   'r': 'escrita por',
   'o': 'Miguel de Cervantes Saavedra'},
  {'f': 'Don Quijote de la Mancha',
   'r': 'publicada su primera parte con el título de',
   'o': 'El ingenioso hidalgo don Quijote de la Mancha'},
  {'f': 'Don Quijote de la Mancha',
   'r': 'es la obra más destacada de',
   'o': 'Literatura española'},
  {'f': 'Don Quijote de la Mancha',
   'r': 'es una de las principales de',
   'o': 'Literatura universal'},
  {'f': 'Don Quijote de la Mancha',
   'r': 'su continuación apareció con el título de',
   'o': 'Segunda parte del ingenioso caballero don Quijote de la Mancha'},
  {'f': 'El ingenioso hidalgo don Quijote de la Mancha',
   'r': 'se publicó dividido en',
   'o': 'Partes'}],
 'conceptos': ['Don Quijote de la Mancha',
  'Novela',
  'Migu

# Bibliografía

* [Unifying Large Language Models and Knowledge Graphs: A Roadmap](https://arxiv.org/abs/2306.08302) de Shirui Pan, et al.
* [The Theory Underlying Concept Maps and How
to Construct and Use Them](https://cmap.ihmc.us/publications/researchpapers/theoryunderlyingconceptmaps.pdf) de Joseph D. Novak y Alberto J. Cañas.